In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

Đọc dữ liệu từ file csv

In [2]:
df = pd.read_csv('/home/vitquay1708/Study_Space/ml/week_1/data/multinomial_nb_orders.csv')
print(df.head(5))

      cuisine     city device user_segment    promo fav_ingredient  \
0    american  city_16    web          new  promo_1          onion   
1        thai  city_10    ios       family  promo_7       cilantro   
2    american  city_18    ios      student  promo_1         cheese   
3    japanese  city_10    web       family  promo_3         ginger   
4  vietnamese  city_10    web       family     none     fish_sauce   

  spice_level day_of_week price_bucket  
0         hot         Wed         high  
1        mild         Wed          mid  
2         hot         Fri          mid  
3        mild         Tue         high  
4        mild         Wed          low  


Tính xác suất P(c) = Nc/N

In [3]:
Y = df['cuisine']
y_labels = np.zeros((len(set(Y)), 2), dtype=Y.dtype)
i = 0
for label in set(Y):
  y_labels[i, 0] = label
  y_labels[i, 1] = (Y == label).sum() / len(Y)
  i+=1


Tính toán xác suất P(xi|k) cùng với phương pháp Laplace Smoothing

In [5]:
def get_unique_value_in_col(df: pd.DataFrame):
  #Lấy ra tên của các trường trong DataFrame
  col_names = df.columns.values
  #Tạo một mảng 2 chiều lưu trữ tên cột và số giá trị phân biệt của cột đó
  unique_values = np.zeros((len(col_names), 2), dtype=col_names.dtype)
  #Đặt cột đầu tiên là tên của các cột
  unique_values[:, 0] = col_names

  #Lấy ra số lượng các giá trị phân biệt trong từng cột rồi gán vào cột thứ 2
  for i in range(len(col_names)):
    unique = set(df[col_names[i]])
    unique_values[i, 1] = len(unique)

  return unique_values

num_labels_M = get_unique_value_in_col(df=df)
print(num_labels_M)

[['cuisine' 10]
 ['city' 20]
 ['device' 5]
 ['user_segment' 6]
 ['promo' 9]
 ['fav_ingredient' 18]
 ['spice_level' 5]
 ['day_of_week' 7]
 ['price_bucket' 4]]


In [7]:
def pxik_for_feature_each_class(X, xi, num_nomials_M, alpha = 1.0):
  X = np.array(X)
  count = (X==xi).sum()
  return (count + alpha)/(len(X) + num_nomials_M*alpha)

Dự đoán output đầu ra

In [9]:
def predict_output_label(df, X, x_input, p_labels, target_feature):
  num_labels_M = get_unique_value_in_col(df)
  #Mảng p chứa loga của xác suất tiên nghiệm
  p = np.log(np.array(p_labels[:, 1], dtype=float))
  for k in range(len(p)):
    #Xk[c] chứa toàn bộ dữ liệu thuộc lớp c
    Xk = X[X[target_feature] == p_labels[k, 0]]
    for i in range(len(x_input)):
      #Công thức xác suất dự đoán sử dụng Laplace Smoothing
      p[k] += np.log(pxik_for_feature_each_class(Xk.iloc[:, i+1], x_input.iloc[i], num_labels_M[i + 1, 1]))
  
  y_star = np.argmax(p)
  return p_labels[y_star, 0]

Chia train test

In [16]:
train_size = round(len(df) * 0.7)
target_col = 'cuisine'
X_train = df[:train_size]
y_train = df[target_col][:train_size]

X_test = df[train_size:].drop(target_col,axis=1).reset_index(drop=True, inplace=False)
y_test = df[target_col][train_size:].reset_index(drop=True, inplace=False)

y_pred = np.zeros(len(y_test), dtype=y_test.dtype)

for i in range(len(X_test)):
  y_pred[i] = predict_output_label(df, X_train, X_test.iloc[i, :], y_labels, target_feature=target_col)



In [18]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred, y_test))
print(predict_output_label(df, X_train, X_train.iloc[3, 1:], y_labels, target_col))

0.49
chinese
